In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [18]:
artists = pd.read_parquet('https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Artist.parquet.gzip')
artists.head()

,id,name,url,summary,picture,birthplace,deathplace,birthdate,deathdate,cause_of_death
0,0,Vincent Van Gogh,http://wikigallery.org/wiki/artist36933/Vincen...,Vincent Willem van Gogh (Dutch: [ˈvɪnsɛnt ˈʋɪl...,0.0,0.0,342.0,1853-03-30,1890-07-29,Gunshot wound
1,1,Pierre Auguste Renoir,http://wikigallery.org/wiki/artist39254/Pierre...,Pierre-Auguste Renoir (French: [pjɛʁ oɡyst ʁən...,1.0,1.0,343.0,1841-02-25,1919-12-03,None
2,2,Claude Oscar Monet,http://wikigallery.org/wiki/artist39249/Claude...,"Oscar-Claude Monet (UK: , US: , French: [klod ...",2.0,2.0,344.0,1840-11-14,1926-12-05,None
3,3,Fernando Botero,http://wikigallery.org/wiki/artist37052/Fernan...,Fernando Botero Angulo (born 19 April 1932) is...,3.0,3.0,NaN,1932-04-19,None,None
4,4,Jean-Léon Gérôme,http://wikigallery.org/wiki/artist46453/Jean-L...,Jean-Léon Gérôme (11 May 1824 – 10 January 190...,4.0,4.0,2.0,1824-05-11,1904-01-10,None


<h3>Artist</h3>

The 'Artist' table is a central component of the dataset and provides information on famous painters and their careers. Next you'll find some basic info about this part of the dataset.

In [19]:
# Select all artists that have death date as NaN
artists_alive = artists[artists['deathdate'].isna()]
print("Potentially alive: " + str(artists_alive.shape))
# Select all artists that have 'deathdate' different from NaN
artists_dead = artists[artists['deathdate'].notna()]
print("Potentially dead: " + str(artists_dead.shape))

Potentially alive: (346, 10)
Potentially dead: (270, 10)


In [20]:
artists_with_birthdate = artists[artists['birthdate'].notna()]
print("artists_with_birthdate: " + str(artists_with_birthdate.shape))

artists_without_birthdate = artists[artists['birthdate'].isna()]
print("artists_without_birthdate: " + str(artists_without_birthdate.shape))

artists_without_birthdate["name"].head()

artists_with_birthdate: (225, 10)
artists_without_birthdate: (391, 10)


5                Paul Cezanne
7         John Singer Sargent
10                    Raphael
11    Michelangelo Buonarroti
16          Peter Paul Rubens
Name: name, dtype: object

In [21]:
artists_without_birthdate_or_deathdate = artists[
    artists['birthdate'].isna() | artists['deathdate'].isna()]

print("artists_without_birthdate_or_deathdate: " + str(artists_without_birthdate_or_deathdate.shape))

artists_without_birthdate_or_deathdate: (400, 10)


It seems that there are so many artists that have no birthdate and no deathdate most of the artists in this dataset don't have a birthdate and a deathdate. Can we fill in the missing birthdates?

In [22]:
# get the name of the first 10 artists without birthdate
artists_without_birthdate_or_deathdate[['name','birthdate','deathdate']].head(10)

,name,birthdate,deathdate
3,Fernando Botero,1932-04-19,None
5,Paul Cezanne,None,None
7,John Singer Sargent,None,None
10,Raphael,None,None
11,Michelangelo Buonarroti,None,1564-02-18
16,Peter Paul Rubens,None,None
22,Tiziano Vecellio,None,1576-08-27
25,Albrecht Durer,None,None
28,Jacques Louis David,None,None
30,Giotto Di Bondone,None,1337-01-08


In [23]:
import scraper_functions as sf
# # Example usage
# convert the artists_without_birthdate_and_deathdate["name"] to a list
a = artists_without_birthdate["name"].tolist()

wikipedia_links, artists_not_found = sf.get_wikipedia_links(a)

IntProgress(value=0, max=391)

In [24]:
# for each element in dictionary, write the key and value to a csv file
with open("artists_without_birthdate_and_deathdate.txt", "w") as file:
    file.write("name,link"+"\n")
    for key, value in wikipedia_links.items():
            file.write(key+","+value+"\n")


In [25]:
names_and_dates = sf.get_dates_from_wikipedia(wikipedia_links)

IntProgress(value=0, max=376)

In [26]:
# for each element in dictionary, write the key and value to a csv file
with open("artists_birthdate_deathdate.txt", "w") as file:
    file.write("name,birthdate,deathdate "+"\n")
    for key, value in names_and_dates.items():
            file.write(key+","+str(value['birth_year'])+","+str(value['death_year'])+"\n")

In [28]:
df = pd.read_csv('artists_birthdate_deathdate.txt', delimiter = ",", encoding ="ANSI")
df.head(10)

ParserError: Error tokenizing data. C error: Expected 3 fields in line 46, saw 4


,id,name,url,summary,picture,birthplace,deathplace,birthdate,deathdate,cause_of_death
81,81,(after) Hieronymus Bosch,https://www.wikigallery.org//wiki/artist64339/...,"Hieronymus Bosch (, Dutch: [ɦijeːˈroːnimʏz ˈbɔ...",81.0,52.0,52.0,None,1516-08-09,None


In [32]:
len(wikipedia_links)

376